In [1]:
%run loading_data.ipynb

audio : 55
audio_only : 33
video : 44
texte : 55
texte_only : 33


#### chargement des données et des features

In [2]:
label_normalised_origin = pd.read_csv('./data/label_normalised.csv', sep=",", index_col=0, header=0)
label_normalised_origin = label_normalised_origin.drop('\t',axis=1)
label_normalised, noteurs_normalised  = transform_annotation(label_normalised_origin)

graded = label_normalised[(label_normalised['isRated'] == 1)].copy()

g_audio, g_audio_only = get_media_type(graded, noteurs_normalised, media='audio')

audio : 55
audio_only : 54


In [3]:
selected_features = ['SR','SNR','CONF','VBR','SCORE']
selected_columns = ['SCENE'] + selected_features
selected_y = 'mean'

audio_features = pd.read_csv('./data/silence_rolling_mean_new.csv', sep='§', engine='python', index_col=0, encoding='utf-8')

DF = pd.merge(audio_features[selected_columns], g_audio_only.drop('count',axis=1), 
              left_on='SCENE', right_on='doc').sort_values(by=['SCENE'])
DF.reset_index(drop=1, inplace=True)
DF.set_index('doc',inplace=True)

for c in selected_features:
    DF.loc[np.isnan(DF.loc[:,c]),c] = np.nanmean(DF.loc[:,c])

In [4]:
DF

,SCENE,SR,SNR,CONF,VBR,SCORE,mean
doc,,,,,,,
100_1,100_1,0.506749,0.975847,0.891960,0.937432,0.533405,-0.187638
107_7,107_7,0.875052,0.992874,0.892274,0.996106,0.823813,0.542397
108_3,108_3,0.610753,0.950284,0.892878,0.992284,0.532751,0.079944
116_15,116_15,0.797808,0.998813,0.889664,0.999472,0.253275,0.492671
128_5,128_5,0.636332,0.998066,0.892274,0.989983,0.471204,1.468359
129_5,129_5,0.613021,0.999324,0.866397,0.785508,0.787136,1.050044
12_13,12_13,0.654612,0.868138,0.892274,0.831517,0.544622,-1.060346
13_2,13_2,0.784851,0.995663,0.892274,0.932309,0.620370,0.613391
147_1,147_1,0.763903,0.998051,0.892274,0.936578,0.746070,1.033869


## Régression linéaire, leave on out

In [5]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

scaler = preprocessing.StandardScaler(with_mean=True, with_std=True, copy=True)
scaler.fit(DF[selected_features])
DF_scaled = pd.DataFrame(scaler.transform(DF[selected_features]), 
                         index = DF.index, columns = selected_features)
DF_scaled[selected_y] = DF[selected_y]

X = DF_scaled[selected_features].values
y = DF_scaled[selected_y].values
preds_test = np.zeros(len(X))

loo = LeaveOneOut()
loo.get_n_splits(X)

coefficients = np.zeros([len(X),len(selected_features)+1])

for i,(train_index, test_index) in enumerate(loo.split(X)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    reg = LinearRegression().fit(X_train, y_train)
    preds_test[i] = reg.predict(X_test)
    coefficients[i,:] = [reg.intercept_] + reg.coef_.tolist()

erreurs = y - preds_test
rmse_test = np.sqrt(np.mean(erreurs**2))
rmse_test

1.0566757563960483

In [6]:
coefficients.mean(axis=0)

array([ 0.28163746,  0.05150939,  0.0805899 , -0.01813273, -0.18506554,
        0.3336484 ])